In [1]:
import networkx as nx

from regraph import Rule, plot_rule
from regraph.neo4j.hierarchy import Neo4jHierarchy
import regraph.neo4j.cypher_utils as cypher

## Initializing Neo4j database

0. When installing neo4j you will be asked to choose login/password for you dbs (here its "neo4j"/"admin"), if you choose other -- change in the cell below.
1. To start neo4j server run `sudo service neo4j start`
2. Check status by running `sudo service neo4j status`. Here you can check the _bolt_ port, change the cell below if different from 7687
3. You can query the db by using the neo4j browser, the address can be found also in the result of 'status', e.g. "Remote interface available at http://localhost:7474/".

In [2]:
# initialize the neo4j driver, wrappped into Neo4jGraph object
h = Neo4jHierarchy(uri="bolt://localhost:7687", user="neo4j", password="admin")

In [3]:
# here we clear the hierarchy
h.clear()
h.drop_all_constraints()

## Adding a graph to the hierarchy

In [4]:
h.add_graph('graphB')
b = h.access_graph('graphB')
nodes = [
    ("a", {"shape": ["circle"], "color": ["white"]}),
    ("b", {"shape": ["circle"], "color": ["black"]}),
    ("c", {"shape": ["square"], "color": ["white", "black"]})
]
edges = [
    ("a", "b"),
    ("b", "c")
]
b.add_nodes_from(nodes)
b.add_edges_from(edges)

In [5]:
h.add_graph('graphC')
c = h.access_graph('graphC')
nodes = [
    ("a", {"shape": ["square"], "color": ["white"]}),
    ("b", {"shape": ["square"], "color": ["black"]}),
    ("c", {"shape": ["circle"], "color": ["white", "black"]})
]
edges = [
    ("a", "b"),
    ("c", "b"),
    ("c", "c")
]
c.add_nodes_from(nodes)
c.add_edges_from(edges)

In [6]:
h.add_graph('graphD')
d = h.access_graph('graphD')
nodes = [
    ("a", {"shape": ["circle"], "color": ["white", "black"]}),
    ("b", {"shape": ["square"], "color": ["white", "black"]})
]
edges = [
    ("a", "b"),
    ("a", "a"),
    ("b", "b")
]
d.add_nodes_from(nodes)
d.add_edges_from(edges)

## Typing

In [7]:
mappingBD = {
    "a":"a",
    "b":"a",
    "c":"b"
}
h.add_typing('graphB', 'graphD', mappingBD)
mappingCD = {
    "a":"b",
    "b":"b",
    "c":"a"
}
h.add_typing('graphC', 'graphD', mappingCD)

## Pullback

In [8]:
h.pullback('graphB', 'graphC', 'graphD', 'graphA')

OPTIONAL MATCH (n:graphB)-[:typing]->(:graphD)<-[:typing]-(m:graphC)

//Perform the intersection of the properties of n, m
WITH [] as new_props, m, n
WITH new_props + REDUCE(pairs = [], k in keys(n) | 
	CASE WHEN ALL(others in [m] WHERE k in keys(others))
	THEN
		pairs + REDUCE(inner_pairs = [], v in n[k] | 
			CASE WHEN ALL(others in [m] WHERE v in others[k])
			THEN
				inner_pairs + {key: k, value: v}
			ELSE
				inner_pairs
			END)
	ELSE
		pairs
	END) as new_props, m, n
WITH apoc.map.groupByMulti(new_props, 'key') as new_props, m, n
WITH apoc.map.fromValues(REDUCE(pairs=[], k in keys(new_props) | 
	pairs + [k, REDUCE(values=[], v in new_props[k] | 
		values + CASE WHEN v.value IN values THEN [] ELSE v.value END)])) as new_props, m, n
CREATE (new_node_a:graphA) 
SET new_node_a.id = toString(id(new_node_a)) 
WITH toString(id(new_node_a)) as id_var , m, new_props, new_node_a, n
SET new_node_a += new_props
WITH n, m, new_node_a 
MERGE (new_node_a)-[:typing {  }]->(n)

MERGE (new_node_a